무정보탐색 알고리즘들의 동작 방식을 이해해 봅시다. 여기 제공하는 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

# 문제 정의
가장 먼저 해결할 문제를 정의합시다.

In [1]:
import math
import heapq
import sys
from collections import defaultdict, deque

## 문제 클래스

In [2]:
class Problem:
    """해결할 문제에 대한 추상 클래스
    다음 절차에 따라 이 클래스를 활용하여 문제해결하면 됨.
    1. 이 클래스의 서브클래스 생성 (이 서브클래스를 편의상 YourProblem이라고 하자)
    2. 다음 메쏘드들 구현
       - actions
       - result
       - 필요에 따라 h, __init__, is_goal, action_cost도
    3. YourProblem의 인스턴스를 생성
    4. 다양한 탐색 함수들을 사용해서 YourProblem을 해결"""

    def __init__(self, initial=None, goal=None, **kwds):
        """초기 상태(initial), 목표 상태(goal) 지정.
        필요에 따라 다른 파라미터들 추가"""
        self.__dict__.update(initial=initial, goal=goal, **kwds)  # __dict__: 객체의 속성 정보를 담고 있는 딕셔너리

    def actions(self, state):
        """행동: 주어진 상태(state)에서 취할 수 있는 행동들을 리턴함.
        대개 리스트 형태로 리턴하면 될 것임.
        한꺼번에 리턴하기에 너무 많은 행동들이 있을 경우, yield 사용을 검토할 것."""
        raise NotImplementedError

    def result(self, state, action):
        """이행모델: 주어진 상태(state)에서 주어진 행동(action)을 취했을 때의 결과 상태를 리턴함.
        action은 self.actions(state) 중 하나여야 함."""
        raise NotImplementedError

    def is_goal(self, state):
        """목표검사: 상태가 목표 상태이면 True를 리턴함.
        상태가 self.goal과 일치하는지 혹은 self.goal이 리스트인 경우 그 중의 하나인지 체크함.
        더 복잡한 목표검사가 필요할 경우 이 메쏘드를 오버라이드하면 됨."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def action_cost(self, state1, action, state2):
        """행동 비용: state1에서 action을 통해 state2에 이르는 비용을 리턴함.
        경로가 중요치 않은 문제의 경우에는 state2만을 고려한 함수가 될 것임.
        현재 구현된 기본 버전은 모든 상태에서 행동 비용을 1로 산정함."""
        return 1

    def h(self, node):
        """휴리스틱 함수:
        문제에 따라 휴리스틱 함수를 적절히 변경해줘야 함."""
        return 0
    
    def __str__(self):
        return f'{type(self).__name__}({self.initial!r}, {self.goal!r})'

    
def is_in(elt, seq):
    """elt가 seq의 원소인지 체크.
    (elt in seq)와 유사하나 ==(값의 비교)이 아닌 is(객체의 일치 여부)로 비교함."""
    return any(x is elt for x in seq)

## 노드

In [3]:
class Node:
    """탐색 트리의 노드. 다음 요소들로 구성됨.
    - 이 노드에 대응되는 상태(한 상태에 여러 노드가 대응될 수도 있음)
    - 이 노드를 생성한 부모에 대한 포인터
    - 이 상태에 이르게 한 행동
    - 경로 비용(g)
    이 클래스의 서브클래스를 만들 필요는 없을 것임."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """parent에서 action을 취해 만들어지는 탐색 트리의 노드 생성"""
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self):
        return f"<{self.state}>"

    def __len__(self): # 탐색 트리에서 이 노드의 깊이
        return 0 if self.parent is None else (1 + len(self.parent))
    
    def __lt__(self, other):
        return self.path_cost < other.path_cost

In [4]:
failure = Node('failure', path_cost=math.inf) # 알고리즘이 해결책을 찾을 수 없음을 나타냄
cutoff  = Node('cutoff',  path_cost=math.inf) # 반복적 깊이 증가 탐색이 중단(cut off)됐음을 나타냄

def expand(problem, node):
    """노드 확장: 이 노드에서 한 번의 움직임으로 도달 가능한 자식 노드들을 생성하여 yield함"""
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    """루트 노드에서부터 이 노드까지 이르는 행동 시퀀스. 
    결국 node가 목표 상태라면 이 행동 시퀀스는 해결책임.
    목표 상태 발견 후 리턴할 행동 시퀀스 생성을 위해 사용됨.
    부모 포인터를 역으로 추적하여 시퀀스 생성"""
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    """루트 노드에서부터 이 노드까지 이르는 상태 시퀀스"""
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

## 경계 자료구조로 사용될 큐

In [5]:
# FIFO Queue
FIFOQueue = deque

# LIFO Queue(Stack)
LIFOQueue = list

# 우선순위 큐
class PriorityQueue:
    """우선순위 큐:
    key 값이 가장 적은 항목이 가장 먼저 삭제되는 큐."""

    def __init__(self, items=(), key=lambda x: x): 
        """key 값 계산하는 함수. 예: best first search의 평가 함수 f"""
        self.key = key
        self.items = [] # (키, 항목) 쌍이 저장되는 힙(heap)
        for item in items:
            self.add(item)
         
    def add(self, item):
        """item 삽입"""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """키 값이 가장 적은 항목을 삭제하여 그 값을 리턴."""
        return heapq.heappop(self.items)[1]
    
    def top(self):
        """키 값이 가장 적은 항목의 값을 조회."""
        return self.items[0][1]

    def __len__(self):
        """항목 개수 조회"""
        return len(self.items)

# 무정보 탐색 알고리즘

## BFS

In [6]:
def breadth_first_search(problem):
    """너비 우선 탐색"""
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

## DFS

In [7]:
def is_cycle(node, k=30):
    "node가 길이 k이하인 싸이클을 형성하는가?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)


def depth_first_recursive_search(problem, node=None):
    """깊이 우선 탐색"""
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

## 깊이 제한 탐색

In [8]:
def depth_limited_search(problem, limit=10):
    """깊이 제한 탐색"""
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

## 반복적 깊이 증가 탐색(IDS)

In [9]:
def iterative_deepening_search(problem):
    """반복적 깊이 증가 탐색"""
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

# 탐색을 통한 문제 해결 예: 루마니아 여행

## 문제 정의

In [10]:
class Map:
    """2차원 지도를 표현하는 그래프.
    Map(links, locations) 형태로 생성.
    - links: [(v1, v2)...] 형식 또는 {(v1, v2): distance...} dict 구조 가능
    - locations: {v1: (x, y)} 형식으로 각 노드의 위치(좌표) 설정 가능
    - directed=False이면, 양방향 링크 추가. 즉, (v1, v2) 링크에 대해 (v2, v1) 링크 추가"""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'):
            links = {link: 1 for link in links}  # 거리 기본값을 1로 설정
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)  # 인접 리스트
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs):
    """(key, val) 쌍들이 주어지면, {key: [val,...]} 형태의 dict 생성하여 리턴."""
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [11]:
class RouteProblem(Problem):
    """지도(Map) 상의 위치 간의 이동 경로를 알아 내는 문제.
    RouteProblem(초기상태, 종료상태, map=Map(...)) 형식으로 문제 생성.
    상태: Map 그래프의 노드들(예: 'A' - 위치 이름이 'A'),
    행동: 목적지 상태들(예: 'A' - 위치 'A'로 이동하는 행동)"""
    
    def actions(self, state): 
        """state에 인접한 장소들"""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """지도 상에서 가능하다면, action 위치로 이동.
        불가능하다면, 위치(상태) 변화 없음. 즉, 위치(상태)는 그대로"""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """s에서 s1로 이동하는 비용: 거리"""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "node의 상태와 목표 상태 사이의 직선 거리"
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "두 점 사이의 직선 거리"
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [12]:
# 루마니아 지도 정의
romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})

In [13]:
# 루마니아 여행 문제 정의
r0 = RouteProblem('A', 'A', map=romania)
r1 = RouteProblem('A', 'B', map=romania)  # 초기상태: A, 목표상태: B
r2 = RouteProblem('N', 'L', map=romania)
r3 = RouteProblem('E', 'T', map=romania)
r4 = RouteProblem('O', 'M', map=romania)

## 무정보 탐색 결과

In [14]:
path_states(breadth_first_search(r1))

['A', 'S', 'F', 'B']

In [15]:
path_states(depth_first_recursive_search(r1))

['A', 'Z', 'O', 'S', 'R', 'C', 'P', 'B']

In [16]:
path_states(depth_limited_search(r1, limit=3))

['A', 'S', 'F', 'B']

In [17]:
path_states(iterative_deepening_search(r1))

['A', 'S', 'F', 'B']

In [18]:
for problem in [r0, r1, r2, r3, r4]:
    print(path_states(iterative_deepening_search(problem)))

['A']
['A', 'S', 'F', 'B']
['N', 'I', 'V', 'U', 'B', 'P', 'C', 'D', 'M', 'L']
['E', 'H', 'U', 'B', 'F', 'S', 'A', 'T']
['O', 'S', 'R', 'C', 'D', 'M']
